# ConversationTokenBufferMemory

`ConversationTokenBufferMemory` 는 최근 대화의 히스토리를 버퍼를 메모리에 보관하고, 대화의 개수가 아닌 **토큰 길이** 를 사용하여 대화내용을 플러시(flush)할 시기를 결정합니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

- `max_token_limit`: 대화 내용을 저장할 최대 토큰의 길이를 설정합니다.


In [ ]:
# from langchain.memory import ConversationTokenBufferMemory
# from langchain_openai import ChatOpenAI


# # LLM 모델 생성
# llm = ChatOpenAI(model_name="gpt-4.1-mini")

# # 메모리 설정
# memory = ConversationTokenBufferMemory(
#     llm=llm, max_token_limit=150, return_messages=True  # 최대 토큰 길이를 50개로 제한
# )

import tiktoken
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# 1. LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 2. Tokenizer (OpenAI 계열 모델에 맞는 기본 토크나이저)
enc = tiktoken.encoding_for_model("gpt-4.1-mini")

# 3. 토큰 기반 메모리 설정

history = InMemoryChatMessageHistory()

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="messages"),
])

chain = prompt | llm

# memory = ConversationTokenBufferMemory(
#     llm=llm, max_token_limit=150, return_messages=True  # 최대 토큰 길이를 50개로 제한
# )

MAX_TOKEN_LIMIT = 150  # 최대 토큰 수 설정
def add_message_with_token_limit(history_obj, message):
    #토큰 수가 MAX_TOKEN_LIMIT를 넘지 않도록 앞에서부터 잘라냄.
    history_obj.add_message(message)

    # 전체 메시지를 하나의 문자열로 변환
    def count_tokens(msgs):
        text = ""
        for m in msgs:
            text += m.content + "\n"
        return len(enc.encode(text))

    # 토큰이 너무 많으면 앞 메시지 제거
    while count_tokens(history_obj.messages) > MAX_TOKEN_LIMIT:
        history_obj.messages.pop(0)   # 가장 오래된 메시지 제거

conversation = RunnableWithMessageHistory(
    chain,
    lambda session_id: history,
    input_messages_key="messages",
    history_messages_key="messages"
)

c:\Users\jeffr\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-SGRq4Tjy-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


임의의 대화를 추가합니다.


In [3]:
# memory.save_context(
#     inputs={
#         "human": "안녕하세요, 저는 최근에 여러분 회사의 공작 기계를 구매했습니다. 설치 방법을 알려주실 수 있나요?"
#     },
#     outputs={
#         "ai": "안녕하세요! 구매해 주셔서 감사합니다. 해당 기계 모델 번호를 알려주시겠어요?"
#     },
# )
# memory.save_context(
#     inputs={"human": "네, 모델 번호는 XG-200입니다."},
#     outputs={
#         "ai": "감사합니다. XG-200 모델의 설치 안내를 도와드리겠습니다. 먼저, 설치할 장소의 전원 공급 상태를 확인해주세요. 기계는 220V 전원이 필요합니다."
#     },
# )
# memory.save_context(
#     inputs={"human": "전원은 확인했습니다. 다음 단계는 무엇인가요?"},
#     outputs={
#         "ai": "좋습니다. 다음으로, 기계를 평평하고 안정된 바닥에 배치해 주세요. 이후, 제공된 사용자 매뉴얼에 따라 케이블 연결을 진행해 주시기 바랍니다."
#     },
# )
# memory.save_context(
#     inputs={"human": "연결은 어떻게 하나요?"},
#     outputs={
#         "ai": "매뉴얼의 5페이지를 참조해 주세요. 케이블 연결에 관한 상세한 지침이 있습니다. 이 과정에서 어려움이 있으시면 추가적으로 도와드리겠습니다."
#     },
# )
# memory.save_context(
#     inputs={"human": "설치가 완료되면 어떻게 해야 하나요?"},
#     outputs={
#         "ai": "설치가 완료되면, 전원을 켜고 초기 구동 테스트를 진행해 주시기 바랍니다. 테스트 절차는 매뉴얼의 10페이지에 설명되어 있습니다. 만약 기계에 이상이 있거나 추가적인 지원이 필요하시면 언제든지 연락 주시기 바랍니다."
#     },
# )
# memory.save_context(
#     inputs={"human": "감사합니다, 도움이 많이 되었어요!"},
#     outputs={
#         "ai": "언제든지 도와드릴 준비가 되어 있습니다. 추가적인 질문이나 지원이 필요하시면 언제든지 문의해 주세요. 좋은 하루 되세요!"
#     },
# )

from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage(content="안녕하세요, 저는 최근에 여러분 회사의 공작 기계를 구매했습니다. 설치 방법을 알려주실 수 있나요?"),
    AIMessage(content="안녕하세요! 구매해 주셔서 감사합니다. 해당 기계 모델 번호를 알려주시겠어요?"),

    HumanMessage(content="네, 모델 번호는 XG-200입니다."),
    AIMessage(content="감사합니다. XG-200 모델의 설치 안내를 도와드리겠습니다. 먼저, 설치할 장소의 전원 공급 상태를 확인해주세요. 기계는 220V 전원이 필요합니다."),

    HumanMessage(content="전원은 확인했습니다. 다음 단계는 무엇인가요?"),
    AIMessage(content="좋습니다. 다음으로, 기계를 평평하고 안정된 바닥에 배치해 주세요. 이후, 제공된 사용자 매뉴얼에 따라 케이블 연결을 진행해 주시기 바랍니다."),

    HumanMessage(content="연결은 어떻게 하나요?"),
    AIMessage(content="매뉴얼의 5페이지를 참조해 주세요. 케이블 연결에 관한 상세한 지침이 있습니다. 이 과정에서 어려움이 있으시면 추가적으로 도와드리겠습니다."),

    HumanMessage(content="설치가 완료되면 어떻게 해야 하나요?"),
    AIMessage(content="설치가 완료되면, 전원을 켜고 초기 구동 테스트를 진행해 주시기 바랍니다. 테스트 절차는 매뉴얼의 10페이지에 설명되어 있습니다. 만약 기계에 이상이 있거나 추가적인 지원이 필요하시면 언제든지 연락 주시기 바랍니다."),

    HumanMessage(content="감사합니다, 도움이 많이 되었어요!"),
    AIMessage(content="언제든지 도와드릴 준비가 되어 있습니다. 추가적인 질문이나 지원이 필요하시면 언제든지 문의해 주세요. 좋은 하루 되세요!")
]


for msg in messages:
    add_message_with_token_limit(history, msg)




최대 토큰의 길이를 **150** 으로 설정하고 대화를 저장했을 때 어떻게 동작하는지 확인해 보겠습니다.


In [4]:
print(history.messages)

[HumanMessage(content='설치가 완료되면 어떻게 해야 하나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='설치가 완료되면, 전원을 켜고 초기 구동 테스트를 진행해 주시기 바랍니다. 테스트 절차는 매뉴얼의 10페이지에 설명되어 있습니다. 만약 기계에 이상이 있거나 추가적인 지원이 필요하시면 언제든지 연락 주시기 바랍니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='감사합니다, 도움이 많이 되었어요!', additional_kwargs={}, response_metadata={}), AIMessage(content='언제든지 도와드릴 준비가 되어 있습니다. 추가적인 질문이나 지원이 필요하시면 언제든지 문의해 주세요. 좋은 하루 되세요!', additional_kwargs={}, response_metadata={})]
